### Back Propagation Through Time

Backpropagation Through Time, or BPTT, is the application of the Backpropagation training algorithm to recurrent neural network applied to sequence data like a time series.

A recurrent neural network is shown one input each timestep and predicts one output.

Conceptually, BPTT works by unrolling all input timesteps. Each timestep has one input timestep, one copy of the network, and one output. Errors are then calculated and accumulated for each timestep. The network is rolled back up and the weights are updated.

<img src="https://raw.githubusercontent.com/ebi-byte/kt/master/NN/BPTT.jpg" style="height: 190px;" />  

                                            Back Propagation Through Time

Spatially, each timestep of the unrolled recurrent neural network may be seen as an additional layer given the order dependence of the problem and the internal state from the previous timestep is taken as an input on the subsequent timestep.
We can summarize the algorithm as follows:  

1. Present a sequence of timesteps of input and output pairs to the network.
2. Unroll the network then calculate and accumulate errors across each timestep.
3. Roll-up the network and update weights.
4. Repeat.

BPTT can be computationally expensive as the number of timesteps increases.


#### Math behind BPTT

There are three weight parameters in RNN: $W$ for hidden state $h$, $U$ for input $x$, and $V$ for output $o$. Let’s examine $V$ first, which turns out to be the easiest case.
Notate the loss as $E_t$ at time step $t$. The loss is a function of $o_t$, and $o_t$ is $Vh_t$ with nonlinearity: $o_t =\Phi (y_t) =\Phi (Vh_t)$. Take the partial derivative of $E_t$ w.r.t $V$:
$$
\begin{align}
\dfrac{\partial E_t}{\partial V} &= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial V}\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial y_t} \dfrac{\partial y_t}{\partial V}\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial y_t} h_t\\
\end{align}
$$
Here, $\dfrac{\partial E_t}{\partial o_t}$ and $\dfrac{\partial o_t}{\partial y_t}$ depends on the specific loss and nonlinearity function being used. The most important thing to notice is, at each time step $t$, $\dfrac{\partial E_t}{\partial V}$ only requires current hidden state $h_t$ ($h_t$ is not a function of $V$). This has no much difference with FeedForward Neural Network. (Note that the objective doesn’t necessarily need to be $V$ which is FFNN; here we assume $V$ for simplicity.)
Next, let’s look at $W$ (also, same for $U$). Take the partial derivative of $E_t$ w.r.t $W$:
$$
\begin{align}
\dfrac{\partial E_t}{\partial W} &= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial W}\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \dfrac{\partial h_t}{\partial W}\\
\end{align}
$$
$\dfrac{\partial E_t}{\partial o_t}$ and $\dfrac{\partial o_t}{\partial h_t}$ depends on the specific loss function used; we can safely put them aside (again, note that in other cases, $\dfrac{\partial o_t}{\partial h_t}$ doesn’t necessarily need to be $V$; it depends on the network structure of the objective). The key thing is to determine $\dfrac{\partial h_t}{\partial W}$.
$h_t = \Phi (Wh_{t-1} + Ux_t)$. Notice that $h_{t-1}$ is also a function of $W$, which in turns depends on $h_{t-2}, …, h_0$, and all of these hidden states are functions of $W$. Therefore, using multivariate chain rule, we can get the following form:
$$
\begin{align}
\dfrac{\partial E_t}{\partial W} &= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \dfrac{\partial h_t}{\partial W}\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \sum_{k=0}^{t} (\dfrac{\partial h_t}{\partial h_k} \dfrac{\partial h_k}{\partial W})
\end{align}
$$
We can then use chain rule on $\dfrac{\partial h_t}{\partial h_k}$ as well.
$$
\begin{align}
\dfrac{\partial E_t}{\partial W} &= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \sum_{k=0}^{t} (\dfrac{\partial h_t}{\partial h_k} \dfrac{\partial h_k}{\partial W})\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \sum_{k=0}^{t} ((\prod_{j=k+1}^{t} \dfrac{\partial h_j}{\partial h_{j-1}}) \dfrac{\partial h_k}{\partial W})\\
&= \dfrac{\partial E_t}{\partial o_t} \dfrac{\partial o_t}{\partial h_t} \sum_{k=0}^{t} ((\prod_{j=k+1}^{t} W \cdot f(h_{j-1})) \dfrac{\partial h_k}{\partial W})
\end{align}
$$
where $f$ is just the derivative of nonlinearity upon hidden state.

If input sequences are comprised of thousands of timesteps, then this will be the number of derivatives required for a single update weight update. This can cause weights to vanish or explode (go to zero or overflow) and make slow learning and model skill noisy.

We can see from above that, at each time step, the backprop needs to compute $(W \cdot f)^{t-k}$ for all previous step $k$. If there is no nonlinearity, $\vert W^{t-k} \vert$ would quickly blow up if $\vert W \vert > 1$, which would make the program crash. This is called exploding gradient problem, and it basically makes the training unpractical.

Same as exploding gradient, vanishing gradient is largely caused by $W^{t-k}$. If $\vert W \vert < 1$, then $\vert W^{t-k} \vert$ would quickly shrink to $0$, especially with sigmoid nonlinearity. The gradient will become $0$ for previous long-distance steps, which makes RNN hard to learn long-range dependencies.


### Truncated Back Propagation Through Time

Truncated Backpropagation Through Time, or TBPTT, is a modified version of the BPTT training algorithm for recurrent neural networks where the sequence is processed one timestep at a time and periodically (k1 timesteps) the BPTT update is performed back for a fixed number of timesteps (k2 timesteps).
 
Truncated backpropagation is arguably the most practical method for training RNNs.

One of the main problems of BPTT is the high cost of a single parameter update, which makes it impossible to use a large number of iterations.

The cost can be reduced with a naive method that splits the 1,000-long sequence into 50 sequences (say) each of length 20 and treats each sequence of length 20 as a separate training case. This is a sensible approach that can work well in practice, but it is blind to temporal dependencies that span more than 20 timesteps.

Truncated BPTT is a closely related method. It processes the sequence one timestep at a time, and every k1 timesteps, it runs BPTT for k2 timesteps, so a parameter update can be cheap if k2 is small. Consequently, its hidden states have been exposed to many timesteps and so may contain useful information about the far past, which would be opportunistically exploited.


<img src="https://raw.githubusercontent.com/ebi-byte/kt/master/NN/TBPTT.jpg" style="height: 190px;" />  

                                        Truncated BackPropagation Through Time

We can summarize the algorithm as follows:

1. Forward pass: Step through the next $k_1$ time steps, computing the input, hidden, and output states.
2. Compute the loss, summed over the previous time steps.
3. Backward pass: Compute the gradient of the loss w.r.t. all parameters, accumulating over the previous $k_2$ time steps (this requires having stored all activations for these time steps). Clip gradients to avoid the exploding gradient problem (happens rarely).
4. Update parameters (this occurs once per chunk, not incrementally at each time step).
5. If processing multiple chunks of a longer sequence, store the hidden state at the last time step (will be used to initialize hidden state for beginning of next chunk). If we've reached the end of the sequence, reset the memory/hidden state and move to the beginning of the next sequence (or beginning of the same sequence, if there's only one).
5. Repeat from step 1.



#### Parameters

The TBPTT algorithm requires the consideration of two parameters:

**k1**: The number of forward-pass timesteps between updates. Generally, this influences how slow or fast training will be, given how often weight updates are performed.

**k2**: The number of timesteps to which to apply BPTT. Generally, it should be large enough to capture the temporal structure in the problem for the network to learn. Too large a value results in vanishing gradients.


Gradient computation and updates are performed every $k_1$ time steps because it's computationally cheaper than updating at every time step. Updating multiple times per sequence (i.e. setting $k_1$ less than the sequence length) can accelerate training because weight updates are more frequent.

Backpropagation is performed for only $k_2$ time steps because it's computationally cheaper than propagating back to the beginning of the sequence (which would require storing and repeatedly processing all time steps). Gradients computed in this manner are an approximation to the 'true' gradient computed over all time steps. But, because of the vanishing gradient problem, gradients will tend to approach zero after some number of time steps; propagating beyond this limit wouldn't give any benefit. Setting $k_2$ too short can limit the temporal scale over which the network can learn. However, the network's memory isn't limited to $k_2$ time steps because the hidden units can store information beyond this period.

Besides computational considerations, the proper settings for $k_1$ and $k_2$ depend on the statistics of the data (e.g. the temporal scale of the structures that are relevant for producing good outputs). They probably also depend on the details of the network. For example, there are a number of architectures, initialization tricks, etc. designed to mitigate the decaying gradient problem.

We will refer to different TBPTT configurations as TBPTT(k1, k2).  
Using this notation, we can define some standard or common approaches:  
Note, here n refers to the total number of timesteps in the sequence:

1. TBPTT(n,n): Updates are performed at the end of the sequence across all timesteps in the sequence (e.g. classical BPTT).
2. TBPTT(1,n): timesteps are processed one at a time followed by an update that covers all timesteps seen so far (e.g. classical TBPTT by Williams and Peng).
3. TBPTT(k1,1): The network likely does not have enough temporal context to learn, relying heavily on internal state and inputs.
4. TBPTT(k1,k2), where k1<k2<n: Multiple updates are performed per sequence which can accelerate training.
5. TBPTT(k1,k2), where k1=k2: A common configuration where a fixed number of timesteps are used for both forward and backward-pass timesteps (e.g. 10s to 100s).

We can see that all configurations are a variation on TBPTT(n,n) that essentially attempt to approximate the same effect with perhaps faster training and more stable results.